Ce notebook illustre notre démarche pour classer les produits sur Rakuten en exploitant les données visuelles. 
La méthodologie s'articule autour de l'extraction de caractéristiques HOG (Histogram of Oriented Gradients) des images et de l'utilisation d'un modèle de Random Forest. Nous avons intégré un rééquilibrage des classes via RandomOverSampler pour adresser le déséquilibre des classes, suivi d'une optimisation des hyperparamètres du modèle via GridSearchCV. 
Cette approche a été choisie pour sa robustesse et son efficacité face aux défis posés par les données images complexes et le déséquilibre des classes

In [2]:
import pandas as pd
import numpy as np
from skimage import io, color, feature, transform
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import RandomOverSampler
import os

In [3]:
# Chemins vers les images et les données CSV
images_path = "C:/Users/user/OneDrive/DATASIENCETEST/PROJET/Data/Update/images/image_train"
X_csv_path = "C:/Users/user/OneDrive/DATASIENCETEST/PROJET/Data/Update/X_train_update.csv"
y_csv_path = "C:/Users/user/OneDrive/DATASIENCETEST/PROJET/Data/Update/Y_train_CVw08PX.csv"

# Chargement des données
X_df = pd.read_csv(X_csv_path, index_col=0)
y_df = pd.read_csv(y_csv_path, index_col=0).squeeze().map(str)
X_df['image_path'] = X_df.apply(lambda row: os.path.join(images_path, f'image_{row.imageid}_product_{row.productid}.jpg'), axis=1)

In [4]:
# Fonction pour extraire les caractéristiques HOG d'une image
def extract_hog_features(image_path):
    image = io.imread(image_path)
    image_gray = color.rgb2gray(image)
    image_resized = transform.resize(image_gray, (128, 64), anti_aliasing=True)
    hog_features = feature.hog(image_resized, pixels_per_cell=(16, 16),
                               cells_per_block=(1, 1), visualize=False)
    return hog_features

In [11]:
# Préparation des données
sample_X = X_df.head(50000)  # Ajustez selon vos besoins
sample_y = y_df.loc[sample_X.index]

features = np.array([extract_hog_features(path) for path in sample_X['image_path']])
labels = sample_y.values

In [12]:
# Rééquilibrage des classes avec RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(features, labels) 

In [13]:
# Division des données rééquilibrées en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [14]:
# Paramètres pour GridSearchCV
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

# Création du modèle RandomForestClassifier pour GridSearchCV
rf_model = RandomForestClassifier(random_state=42)

# Création et exécution de GridSearchCV
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200]},
             scoring='accuracy', verbose=1)

In [15]:
# Meilleurs paramètres et score
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score:", grid_search.best_score_)

Meilleurs paramètres: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
Meilleur score: 0.9084459391024264


In [16]:
# Prédiction et évaluation sur l'ensemble de test
y_pred = grid_search.predict(X_test)
print(f"Précision sur l'ensemble de test: {accuracy_score(y_test, y_pred):.2f}")
print("Rapport de classification :")
print(classification_report(y_test, y_pred))

Précision sur l'ensemble de test: 0.94
Rapport de classification :
              precision    recall  f1-score   support

          10       0.93      0.97      0.95      1178
        1140       0.95      0.97      0.96      1141
        1160       0.98      0.96      0.97      1244
        1180       1.00      1.00      1.00      1180
        1280       0.84      0.79      0.81      1153
        1281       0.99      0.97      0.98      1270
        1300       0.83      0.84      0.83      1192
        1301       1.00      1.00      1.00      1218
        1302       0.96      0.96      0.96      1169
        1320       0.91      0.91      0.91      1189
        1560       0.89      0.81      0.85      1204
        1920       0.94      0.95      0.95      1171
        1940       0.99      1.00      1.00      1235
        2060       0.77      0.83      0.80      1161
        2220       1.00      1.00      1.00      1215
        2280       0.85      0.93      0.89      1165
        2403  

En adoptant cette stratégie avancée de traitement d'images et de rééquilibrage des classes, nous avons obtenu une précision de classification de 94% sur notre ensemble de test. Ces résultats démontrent l'efficacité de notre approche combinée de techniques de traitement d'images, de rééquilibrage des classes, et de modélisation par apprentissage automatique, soulignant l'impact significatif sur la classification précise des produits et l'amélioration potentielle de l'expérience utilisateur sur les plateformes e-commerce.